In [ ]:
import inspect
import os
import sys
import bookutils
import ipywidgets as widgets

In [ ]:
from Tracer import Tracer
from bookutils import input,next_inputs
from pythonfile import py_fun1,py_fun2
import re

In [ ]:
class State:
    def __init__(self,event,abs_lineno,code,changed_vars,call_stack,frame,rel_lineno,fun_name,file):
        self.event = event
        self.abs_lineno = abs_lineno
        self.rel_lineno = rel_lineno
        self.code = code
        self.changed_vars = changed_vars
        self.call_stack = call_stack
        self.frame = frame
        self.stackdepth = None
        self.fun_name = fun_name
        self.file = file

    def __str__(self):
        return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + \
               repr(self.code) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name) + '  ('+ repr(self.file)+')'

In [ ]:
class Breakpoint:
    def __init__(self,type,line=None,function=None,condition=None,file=None,is_active=True):
        self.type=type
        self.line_number=line
        self.func_name=function
        self.cond_expression = condition
        self.file = file
        self.is_active=is_active

    def break_here(self,state,variables,backwards=False):
        if not self.is_active:
            return False

        if self.type == 'line':
            return state.abs_lineno==self.line_number and state.file==self.file
        elif self.type == 'func':
            if backwards:
                return state.event=="return" and state.fun_name==self.func_name
            else:
                return state.event=="call" and state.fun_name==self.func_name
        elif self.type =='cond':
            if state.abs_lineno==self.line_number:
                try:
                    return eval(self.cond_expression,globals(),variables)
                except Exception as err:
                    #self.log(f"{err.__class__.__name__}: {err}")
                    return False
            else:
                return False
        else:
            return False

    def is_active(self):
        return self.is_active

    def enable(self):
        self.is_active=True

    def disable(self):
        self.is_active=False

    def __str__(self):
        if self.type == 'line':
            return f"line  {repr(self.file)}:{self.line_number}  {self.is_active}"
        elif self.type == 'func':
            return f"func  {repr(self.file)}:{self.func_name}  {self.is_active}"
        elif self.type =='cond':
            return f"cond  {repr(self.file)}:{self.line_number}  {self.is_active}  {repr(self.cond_expression)}"
        else:
            return ""

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
class Tracer(object):
    def __init__(self, file=sys.stdout):
        """Trace a block of code, sending logs to file (default: stdout)"""
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        pass

    def changed_vars(self, new_vars):
        changed = {}
        for var_name in new_vars:
            if (var_name not in self.last_vars or
                    self.last_vars[var_name] != new_vars[var_name]):
                changed[var_name] = new_vars[var_name]
        self.last_vars = new_vars.copy()
        return changed

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        # print(*objects, sep=sep, end=end, file=self.file, flush=True)
        # self.logger = self.logger + "\n" + " ".join(objects)


        current_state = State(objects[0], objects[1],objects[2],objects[3],None,objects[5],objects[6],objects[7],objects[8])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function. To be overridden in subclasses."""
        self.log_state(frame, event, arg)
        #self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def log_state(self, frame, event, arg):

        changes = self.changed_vars(frame.f_locals)
        module = inspect.getmodule(frame.f_code)

        if module is None:
            source,function_start = inspect.getsourcelines(frame.f_code)
            file = inspect.getsourcefile(frame)
            relative_lineno = frame.f_lineno-function_start + 1
        else:
            source,function_start = inspect.getsourcelines(module)
            path = inspect.getsourcefile(module)
            _, tail = os.path.split(path)
            file = tail.replace(".py","")

            relative_lineno = frame.f_lineno-function_start

        #TODO remove frame from State object again
        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, changes, None, frame, relative_lineno,frame.f_code.co_name,file)

        if event == 'line':
            current_line = source[relative_lineno-1]
            self.log(event, frame.f_lineno, current_line, changes, None, frame, relative_lineno,frame.f_code.co_name,file)

        if event == 'return':
            self.log(event, frame.f_lineno, repr(arg), changes,None, frame, relative_lineno,frame.f_code.co_name,file)
            self.last_vars = {}  # Delete 'last' variables


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at begin of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        #self.print_log()

    def print_log(self):
        index = 0
        for log in self.loglist:
            #print('Log Entry: ' + str(index))
            #index = index + 1
            print(log)

    def get_log(self):
        return self.loglist

In [ ]:
class Debugger(object):
    def __init__(self):

        self.current_state = None
        self.callstack = None
        self.current_step = 0
        self.stepping = True
        self.breakpoints = dict()
        self.watchpoints = dict()
        self.last_watchpoint = 1
        self.interact = True
        self.variables = {}

    def __enter__(self):
        with Tracer() as t:
            py_fun1(5)
        self.execution_log = t.get_log()
        t.print_log()
        print("\n")
        print("\n")
        print("\n")
        if len(self.execution_log)<1:
            print("Log is empty")
            return
        self.construct_callstack()
        self.current_state=self.execution_log[self.current_step]
        self.print_debugger_status()
        self.interaction_loop()

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

    def step(self):
        if self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            self.construct_vars()
            return True
        else:
            return False

    def step_back(self):
        if self.current_step> 0:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            return True
        else:
            return False


    def construct_callstack(self):

        callstack=list()
        currentdepth = -1

        for i in range(0,len(self.execution_log)):
            currentexecution = self.execution_log[i]

            if currentexecution.event == 'call':
                #self.log(currentexecution)
                callstack.append(currentexecution)
                currentdepth = currentdepth + 1

            elif currentexecution.event == 'return':
                currentdepth = currentdepth - 1

            currentexecution.stackdepth = currentdepth

        self.callstack=callstack

    def backconstruct_vars(self):
        backconstructed_vars = dict()
        #TODO: check if this properly displays function arguments when in 'call' event
        #TODO: does not work if in step 0
        if self.current_step>0:
            num_subroutines = 0
            for i in range(0,self.current_step):
                current_execution = self.execution_log[self.current_step-(i+1)]
                if current_execution.event == 'return':
                    num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0: break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(backconstructed_vars)
                    backconstructed_vars = changed_vars

            self.variables = backconstructed_vars
            return  True
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
            return False

    def construct_vars_at(self,target_step):
        constructed_vars = dict()

        if target_step>0:
            num_subroutines = 0
            for i in range(0,target_step):
                current_execution = self.execution_log[target_step-i]
                if current_execution.event == 'return':
                    if i>0:
                        num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0:
                        changed_vars = current_execution.changed_vars.copy()
                        changed_vars.update(constructed_vars)
                        constructed_vars = changed_vars
                        break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(constructed_vars)
                    constructed_vars = changed_vars
        else:
            self.variables=self.execution_log[0].changed_vars.copy()
        return constructed_vars

    def construct_vars(self):
        event = self.current_state.event
        if event == 'return':
            self.variables.clear() #TODO this deletes too early
        elif event == 'call':
            self.variables = self.current_state.changed_vars.copy()
        else:
            self.variables.update(self.current_state.changed_vars)

    def interaction_loop(self):
        self.interact = True
        while self.interact:
            command = input("(debugger) ")
            self.execute(command)

    def commands(self):
        cmds = [method.replace('_command', '')
                for method in dir(self.__class__)
                if method.endswith('_command')]
        cmds.sort()
        return cmds

    def command_method(self, command):
        if command.startswith('#'):
            return None  # Comment

        possible_cmds = [possible_cmd for possible_cmd in self.commands()
                         if possible_cmd==command]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None

        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')

    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)

    def next_command(self,arg=""):
        """Step over function calls going to the next line"""


        fun_name = self.current_state.fun_name

        if self.current_state.event == 'call':
            while not (self.current_state.event == 'return' and self.current_state.fun_name == fun_name):
                self.step()
                if self.has_breakpoint():
                    self.check_watchpoints()
                    self.print_debugger_status()
                    return
        self.step()
        self.check_watchpoints()
        self.print_debugger_status()

    def previous_command(self,arg=""):
        """Step over function calls going to the previous line"""
        fun_name = self.current_state.fun_name
        self.step_back()
        #TODO don't know if this is the intended functionality
        if self.current_state.event!='call':
            while not (self.current_state.fun_name == fun_name and self.current_state.event=='line'):
                self.step_back()
                if self.has_breakpoint(True):
                    self.check_watchpoints()
                    self.print_debugger_status()
                    return
        else:
            self.step_back()

        self.backconstruct_vars()
        self.check_watchpoints()
        self.print_debugger_status()

    def next_state(self):
        if self.current_step<len(self.execution_log)-1:
            return self.execution_log[self.current_step+1]
        else:
            return None

    def lines_above(self,numLines):

        surrounding_lines = list()
        current_function = self.current_state.fun_name

        candidate_step = self.current_step - 1

        while len(surrounding_lines)<numLines and candidate_step>=0:
            if self.execution_log[candidate_step].event == 'line' or self.execution_log[candidate_step].event == 'return':
                if self.execution_log[candidate_step].fun_name == current_function:
                    surrounding_lines.append(self.execution_log[candidate_step])

            candidate_step = candidate_step -1

        surrounding_lines.reverse()

        return surrounding_lines

    def lines_below(self,numLines):

             print('Num Lines '+str(numLines))
             surrounding_lines = list()
             candidateStep = self.current_step + 1

             current_function = self.current_state.fun_name

             while len(self.execution_log)-candidateStep>0 and len(surrounding_lines)<numLines:
                 if (self.execution_log[candidateStep].event== 'line' or self.execution_log[candidateStep].event== 'return') and self.execution_log[candidateStep].fun_name == current_function:
                     surrounding_lines.append(self.execution_log[candidateStep])
                 elif self.execution_log[candidateStep].event == 'call':
                     surrounding_lines.append(self.execution_log[candidateStep])
                 candidateStep = candidateStep + 1

             return surrounding_lines


    def list_command(self,args=""):

             argiterator = map(int, re.findall(r'\d+', args))
             parsed_args = list()

             for arg in argiterator:
                 print(str(arg))
                 parsed_args.append(arg)

             surrounding_lines = list()

             if len(parsed_args) == 0:
                 surrounding_lines.append(self.lines_above(2))

                 surrounding_lines.append("      >> " + str(self.current_state))

                 surrounding_lines.append(self.lines_below(2))

             elif len(parsed_args) == 1:
                 surrounding_lines = surrounding_lines + (self.lines_above(parsed_args[0]))

                 surrounding_lines = surrounding_lines + [("      >> " + str(self.current_state))]

                 surrounding_lines = surrounding_lines + (self.lines_below(parsed_args[0]))

             elif len(parsed_args) == 2:
                 surrounding_lines  = surrounding_lines + (self.lines_above(parsed_args[0]))

                 surrounding_lines = surrounding_lines + [("      >> " + str(self.current_state))]

                 surrounding_lines  = surrounding_lines + (self.lines_below(parsed_args[1]))

             for surrounding_line in surrounding_lines:
                 print(str(surrounding_line))


    def until_command(self,arg=""):
        """Execute forwards until a certain point.
        'until' executes until a line greater than the current line is reached
        'until <line_number>' executes until a line greater than <line_number> is reached
        'until <filename>:<line_number>' executes until line <line_number> is reached in file <filename>
        'until <function_name>' executes until function <function_name> is called
        'until <filename>:<function_name>' executes until function <function_name> declared in <filename> is called
        """
        #TODO special case no arg act as next
        if not arg:
            #no arg case
            self.until_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else: 
                function_name = secondary_arg

            if target_line > 0:
                self.until_line(target_line,filename)
            else:
                self.until_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.until_function(function_name,self.current_state.file)
            else:
                self.until_line(target_line,self.current_state.file)


    def backuntil_command(self,arg=""):
        """Execute backwards until a certain point.
        'backuntil' executes backwards until a line less than the current line is reached
        'backuntil <line_number>' executes backwards until a line less than <line_number> is reached
        'backuntil <filename>:<line_number>' executes backwards until line <line_number> is reached in file <filename>
        'backuntil <function_name>' executes backwards until function <function_name> is called
        'backuntil <filename>:<function_name>' executes backwards until function <function_name> declared in <filename> is called
        """
        if not arg:
            #no arg case
            self.backuntil_line(self.current_state.abs_lineno,self.current_state.file)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            if secondary_arg.isdigit():
                target_line=int(secondary_arg)
            else:
                function_name = secondary_arg
            if target_line > 0:
                self.backuntil_line(target_line,filename)
            else:
                self.backuntil_function(function_name,filename)
        else:
            #simple arg case
            if arg.isdigit():
                target_line=int(arg)
            else:
                function_name=arg

            if function_name:
                self.backuntil_function(function_name,self.current_state.file)
            else:
                self.backuntil_line(target_line,self.current_state.file)

    def until_function(self,target_function,target_file):
        """Advance until call to target_function and in target_file if given"""
        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()

        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call"
                   and self.current_state.file==target_file ):
            stepped = self.step()
            if not stepped:
                break
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        if stepped:
            self.check_watchpoints()
            self.print_debugger_status()
        else:
            self.log("Couldn't find any execution in " + target_file + " of function named: "+ str(target_function))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()

    def backuntil_function(self,target_function,target_file):
        """Run backwards until call to target_function and in target_file if given"""
        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()

        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call" and
                    (self.current_state.file==target_file)):
            stepped = self.step_back()
            if not stepped:
                break

            self.backconstruct_vars()
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return

        if stepped:
            self.backconstruct_vars() #TODO maybe not needed anymore
            self.check_watchpoints()
            self.print_debugger_status()
        else:
            self.log("Couldn't find any execution in " + target_file + " before current line of function named: "+ str(target_function))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()
        
    def until_line(self,target_line,target_file):
        """Advance until program line is greater than target_line and in target_file if given"""
        if target_line<=0:
            self.log("Invalid line: " + str(target_line))
            return

        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()
        # stepped=False
        # while (self.current_state.abs_lineno <= target_line) or (self.current_state.file!=target_file) :
        #     stepped=self.step()
        #     if not stepped:
        #         break

        found = False
        for i in range(self.current_step,len(self.execution_log)):
            if self.current_state.abs_lineno>target_line and self.current_state.file==target_file:
                found = True
                break
            self.step()
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        if found:
            self.check_watchpoints()
            self.print_debugger_status()
        else:
            self.log("Couldn't reach any line in " + target_file + " greater than "+ str(target_line))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()

    def backuntil_line(self,target_line,target_file):
        """Run backwards until program line is lower than target_line and in target_file if given"""
        if target_line<=1:
            self.log("Invalid line: " + str(target_line))
            return

        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()

        found = False
        for i in range(0,self.current_step+1):
            if self.current_state.abs_lineno<target_line and self.current_state.file==target_file:
                found = True
                break
            self.step_back()
            self.backconstruct_vars()
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return
    
        if found:
            self.backconstruct_vars() #TODO maybe not needed anymore
            #self.construct_vars_at(self.current_step)
            self.check_watchpoints()
            self.print_debugger_status()
        else:
            self.log("Couldn't reach any in " + target_file + " line lower than "+ str(target_line))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()
            

    def finish_command(self,arg=""):
        """Execute until return of current function"""
        fun_name = self.current_state.fun_name
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="return"):
            self.step()
            if self.has_breakpoint():
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.check_watchpoints()
        self.print_debugger_status()

    def start_command(self,arg=""):
        """Execute backwards until a function start"""
        fun_name = self.current_state.fun_name
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="call"):
            self.step_back()
            self.backconstruct_vars()
            if self.has_breakpoint(True):
                self.check_watchpoints()
                self.print_debugger_status()
                return

        self.variables = self.current_state.changed_vars.copy() #TODO what about member variables
        self.check_watchpoints()
        self.print_debugger_status()

    def where_command(self,arg=""):
        currentstate = self.execution_log[self.current_step]
        currentdepth = currentstate.stackdepth
        for i in range (0,len(self.callstack)):
            if i!=currentdepth:
                self.log(str(self.callstack[i]))
            else:
                self.log('     > ' + str(self.callstack[i]))

    def has_breakpoint(self,backwards=False):
        active_breakpoints = [bp for bp in self.breakpoints if self.breakpoints[bp].break_here(self.current_state,self.variables,backwards)]
        return len(active_breakpoints)>0

    def quit_command(self,arg=""):
        """End the debugger session"""
        self.interact=False
        self.log("Quit debugger session")

    def continue_command(self, arg=""):
        """ Continue execution forward until a breakpoint is hit, or the program finishes"""
        stepping = True
        while stepping:
            stepped = self.step()
            if not stepped:
                #self.interact=False
                stepping = False
            if self.has_breakpoint():
                stepping = False

        self.check_watchpoints()
        self.print_debugger_status()

    def reverse_command(self, arg=""):
        while not (self.current_state.abs_lineno in self.breakpoints or self.current_step==0):
                self.step_back()
        #TODO: variables??
        self.check_watchpoints()
        self.print_debugger_status()

    def backstep_command(self,arg=""):
        """Step to the previous executed line"""
        status = self.backconstruct_vars()
        if status:
            self.current_step-=1
            self.current_state = self.execution_log[self.current_step]
        self.check_watchpoints()
        self.print_debugger_status()

    def step_command(self, arg=""):
        """Execute up to the next line"""
        self.step()
        #self.update_watchpoints(True)
        self.check_watchpoints()
        #print("constructed:"+ repr(self.construct_vars_at(self.current_step)))
        self.print_debugger_status()

    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        #vars = self.current_state.frame.f_locals
        vars = self.variables
        if not arg:
            self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, None, vars))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")

    def break_command(self, arg=""):
        """Set a breakpoint in given line. If no line is given, list all breakpoints"""
        if arg:
            if len(self.breakpoints)>0:
                largest_key= max(self.breakpoints.keys())
            else:
                largest_key = 0

            if arg.isdigit():
                self.breakpoints[largest_key+1]=Breakpoint("line",int(arg),None,None,self.current_state.file)
            else:
                sep = arg.find(':')
                if sep>0:
                    file_name = arg[:sep].strip()
                    func_name = arg[sep+1:].strip()
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,func_name,None,file_name)
                else:
                    self.breakpoints[largest_key+1]=Breakpoint("func",None,arg,None,self.current_state.file)
        else:
            self.log("Missing argument, use as 'break <line>|<function_name>|<file_name>:<function_name>'")

    def delete_command(self, arg=""):
        """Delete breakpoint with given <breakpoint_id>"""
        if arg:
            if arg.isdigit():
                try:
                    self.breakpoints.pop(int(arg))
                except KeyError:
                    self.log(f"No such breakpoint: {arg}")
            else:
                self.log("Argument has to be the id of a breakpoint, use as 'delete <breakpoint_id>'")
        #self.log("Breakpoints:", self.breakpoints)

    def breakpoints_command(self,arg=""):
        """Display all available breakpoints"""
        for breakpoint in self.breakpoints:
            self.log(str(breakpoint)+ "   " +str(self.breakpoints[breakpoint]))

    def disable_command(self,arg=""):
        """Use as 'disable <breakpoint_id>'    Suspend  breakpoint with the index <breakpoint_id>"""
        if arg:
            if arg.isdigit():
                try:
                    self.breakpoints[int(arg)].disable()
                except KeyError:
                    pass
            else:
                self.log("Argument has to be integer, use as 'disable <breakpoint_id>'")
        else:
            self.log("Missing argument, use as 'disable <breakpoint_id>'")

    def enable_command(self,arg=""):
        """Use as 'enable <breakpoint_id>'    Re-enable a breakpoint with the index <breakpoint_id>"""
        if arg:
            if arg.isdigit():
                try:
                    self.breakpoints[int(arg)].enable()
                except KeyError:
                    pass
            else:
                self.log("Argument has to be integer, use as 'enable <breakpoint_id>'")
        else:
            self.log("Missing argument, use as 'enable <breakpoint_id>'")

    def cond_command(self,arg=""):
        """Use as 'cond <line> <condition>'    Set a breakpoint at which the execution is stopped at line <line>
        if a condition <condition> is true"""
        if arg:
            sep = arg.find(' ')
            if sep>0:
                line = arg[:sep].strip()
                cond_expression = arg[sep+1:].strip()
                if not line.isdigit():
                    self.log("Faulty argument, use as 'cond <line> <condition>'")
                    return

                if len(self.breakpoints)>0:
                    largest_key= max(self.breakpoints.keys())
                else:
                    largest_key = 0

                self.breakpoints[largest_key+1]=Breakpoint("cond",int(line),None,cond_expression,self.current_state.file)
            else:
                self.log("Faulty argument, use as 'cond <line> <condition>'")
        else:
            self.log("Missing argument, use as 'cond <line> <condition>'")

    def watch_command(self,arg=""):
        """Creates a numbered watchpoint for the given variable"""
        if arg:
            variables = [var for (var, val) in self.watchpoints.values()]
            if not arg in variables:
                if len(self.watchpoints)>0:
                    largest_key= max(self.watchpoints.keys())
                else:
                    largest_key = 0
                self.watchpoints[largest_key+1]=(arg,None)

            else:
                self.log("Watchpoint already exists: "+arg)
        else:
            self.print_watchpoints()

    def unwatch_command(self,arg=""):
        """Remove a watchpoint"""
        if arg:
            if arg.isdigit():
                try:
                    self.watchpoints.pop(int(arg))
                except KeyError:
                    self.log("No such watchpoint: "+ arg)
                self.log("Watchpoints", self.watchpoints)
            else:
                self.log("The argument has to be an integer: unwatch <watch_id>")
        else:
            self.log("Specify which watchpoint to remove: unwatch <watch_id>")
            self.log("Watchpoints", self.watchpoints)

    def update_watchpoints(self,print_changes=False):
        changes={}
        for watch_id in self.watchpoints:
            entry = self.watchpoints[watch_id]
            if entry[0] in self.variables:
                if self.variables[entry[0]]!=entry[1]:
                    changes[watch_id]=(entry[0],self.variables[entry[0]])
        self.watchpoints.update(changes)
        if print_changes and changes:
            #self.log("Changed watchpoints: " + repr(changes))
            changes_s= ", ".join([repr(self.watchpoints[watch_id][0])+" = "+ repr(self.watchpoints[watch_id][1])
                               for watch_id in changes])
            self.log("Changed watchpoints: " + changes_s)
        return changes
        #repr(eval(self.current_state.code, None, self.variables))

    def check_watchpoints(self):
        if not self.watchpoints:
            return

        if self.current_step>0:
            if self.execution_log[self.current_step-1].event == 'return':
                var_backup = self.variables.copy()
                step = self.current_step-1
                state =self.execution_log[step]
                while not (step == 0 or (state.event=='line' and state.fun_name == self.current_state.fun_name)):
                    step -= 1
                    state = self.execution_log[step]
                if step != 0:
                    self.log(str(state))
                    self.variables = self.construct_vars_at(step)
                    self.update_watchpoints()
                    self.variables=var_backup
                    self.update_watchpoints(True)
            else:
                self.update_watchpoints(True)
        else:
            self.update_watchpoints(True)

    def print_watchpoints(self):
        watched_vars= ", ".join([str(watch_id) + ": "+repr(self.watchpoints[watch_id][0])
                               for watch_id in self.watchpoints])
        self.log(watched_vars)


    def print_debugger_status(self):
        frame = self.current_state.frame
        event = self.current_state.event
        changes = self.current_state.changed_vars
        changes_s = ", ".join([var + " = " + repr(changes[var])
                               for var in changes])

        if event == 'call':
            self.log(repr(self.current_state.abs_lineno) + "    Calling " + self.current_state.fun_name + '(' + changes_s + ')')

        if event == 'line':
            current_line = self.current_state.code
            self.log(repr(self.current_state.abs_lineno) + '    ' + current_line)
        if event == 'return':
            self.log(repr(self.current_state.abs_lineno) +'    '+ self.current_state.fun_name + '()' + " returns " + repr(self.current_state.code)) #used to have +repr(arg)


    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        print(*objects, sep=sep, end=end, file=sys.stdout, flush=True)

    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""
        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands()
                             if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                print(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                print(f"Ambiguous command {repr(command)}. Possible expansions are:")
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            print(f"{cmd:10} -- {method.__doc__}")

In [ ]:
def foo():
    a = 12
    a= 13
    a = bar(21)
    x = 5
    #b = fun(a,x)
    a = 11

    return

def bar(arg):
    b = arg
    a = 7
    return b
    #code()

def code():
    return



In [ ]:
with Debugger():
     pass



In [ ]:
#d= Debugger(res)
#d.start()
#next_inputs(["step","print","step","step","print","step","print s","continue"])
#next_inputs(["step","step","step","step","step","step","continue"])
#next_inputs(["step","break 4","continue","continue"])
#with Debugger():
#    remove_html_markup("<b>bold</b>")

In [ ]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))


play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
tester = widgets.HBox([play, slider])


top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = widgets.IntText(description='Top left', layout=widgets.Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=widgets.Layout(width='auto', height='auto'))
bottom_left_slider = widgets.IntSlider(description='Bottom left', layout=widgets.Layout(width='auto', height='auto'))
bottom_right_slider = widgets.IntSlider(description='Bottom right', layout=widgets.Layout(width='auto', height='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

widgets.AppLayout(header=tester,
          center=center_button,
          right_sidebar=right_button,
          pane_widths=[2, 0.5, 1],
          pane_heights=[1, 17, 1])


